First, open an ssh tunnel to the CatSim database hosted by the University of Washington.  Open a terminal window and type

```
ssh -L 51433:fatboy.phys.washington.edu:1433 simsuser@gateway.astro.washington.edu
```

There is some configuration that you will have to do to make sure this works.  Instructions are here:

https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database

In [ ]:
import os
opsimdb = os.path.join("/Users","danielsf","physics")
opsimdb = os.path.join(opsimdb, "lsst_150412", "Development", "garage")
opsimdb = os.path.join(opsimdb, "OpSimData", "kraken_1042_sqlite.db")

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import RRLyStarObj
stardb = RRLyStarObj()

In [ ]:
from lsst.sims.catUtils.utils import StellarLightCurveGenerator
lc_gen = StellarLightCurveGenerator(stardb, opsimdb)

In [ ]:
raRange = (60.0, 65.0)
decRange = (-15.0, -10.0)
bandpass = ('g', 'r', 'i')
pointings = lc_gen.get_pointings(raRange, decRange, bandpass=bandpass)

In [ ]:
print len(pointings)

In [ ]:
lc_dict, truth = lc_gen.light_curves_from_pointings(pointings)

In [ ]:
lc_dict.keys()[-1]

In [ ]:
lc = lc_dict[853673991]

In [ ]:
print len(lc['g']['mjd']), len(lc['r']['mjd']), len(lc['i']['mjd'])

In [ ]:
print truth[853673991]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

In [ ]:
from gatspy.periodic import LombScargleFast
model = LombScargleFast().fit(lc['r']['mjd'], lc['r']['mag'],
                              lc['r']['error'])
periods, power = model.periodogram_auto(nyquist_factor=100)

fig, ax = plt.subplots()
ax.plot(periods, power)
ax.set(xlim=(0.0001, 1.0))

In [ ]:
type(model)


In [ ]:
model.optimizer.period_range = (0.1, 1.4)
best_period = model.best_period
print best_period